In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
pwd

'/Users/nagapurnithinreddy'

In [3]:
cd downloads

/Users/nagapurnithinreddy/Downloads


In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
neighborhoods_data = newyork_data['features']

In [6]:
neighborhoods_data[0:2]

[{'type': 'Feature',
  'id': 'nyu_2451_34572.1',
  'geometry': {'type': 'Point',
   'coordinates': [-73.84720052054902, 40.89470517661]},
  'geometry_name': 'geom',
  'properties': {'name': 'Wakefield',
   'stacked': 1,
   'annoline1': 'Wakefield',
   'annoline2': None,
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.84720052054902,
    40.89470517661,
    -73.84720052054902,
    40.89470517661]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.2',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82993910812398, 40.87429419303012]},
  'geometry_name': 'geom',
  'properties': {'name': 'Co-op City',
   'stacked': 2,
   'annoline1': 'Co-op',
   'annoline2': 'City',
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.82993910812398,
    40.87429419303012,
    -73.82993910812398,
    40.87429419303012]}}]

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(neighborhoods['Borough'].unique()),neighborhoods.shape[0]))

The dataframe has 5 boroughs and 306 neighborhoods.


In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [13]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [14]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [15]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [16]:
CLIENT_ID = 'EZ2E4CDICX3Z4OMNETEHL0IEE4WCLTWE021EEHFGMQHU1OG3' # your Foursquare ID

CLIENT_SECRET = '5AY53EUO4BANNHVOTPT0RKX2DPFRKDUNB3P1NGEUJYOWAWJL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EZ2E4CDICX3Z4OMNETEHL0IEE4WCLTWE021EEHFGMQHU1OG3
CLIENT_SECRET:5AY53EUO4BANNHVOTPT0RKX2DPFRKDUNB3P1NGEUJYOWAWJL


In [17]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [18]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [19]:
# type your answer here
Limit = 100
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    Limit)

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb8360cedbcad001ba3e276'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 27,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4b4429abf964a52037f225e3',
   'name': "Arturo's",
   'location': {'address': '5198 Broadway',
    'crossStreet': 'at 225th St.',
    'lat': 40.87441177110231,
    'lng': -73.91027100981574,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.87441177110231,
      'lng': -73.91027100981574},
     {'label': 'entrance', 'lat': 40.874401, 'lng': -73.910339}],
    'distance': 240,
    'postalCode': '10463',
    'cc': 'US',
    'city': 'New York',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['5198 Broadway (at 225th St.)',
     'New York, NY 10463',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1ca941735',
     'name': 'Pizza Place',
     'pluralName': 'Pizza Places',
     'shortName': 'Pizza',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',

In [23]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.delivery.id,venue.delivery.url,venue.delivery.provider.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.icon.name,venue.photos.count,venue.photos.groups,venue.venuePage.id,venue.location.neighborhood
0,e-0-4b4429abf964a52037f225e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b4429abf964a52037f225e3,Arturo's,5198 Broadway,at 225th St.,40.874412,-73.910271,"[{'label': 'display', 'lat': 40.87441177110231...",240,10463,US,New York,NY,United States,"[5198 Broadway (at 225th St.), New York, NY 10...","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",72548,https://www.seamless.com/menu/arturos-pizza-51...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,0,[],NaN,NaN
1,e-0-4baf59e8f964a520a6f93be3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4baf59e8f964a520a6f93be3,Bikram Yoga,5500 Broadway,230th Street,40.876844,-73.906204,"[{'label': 'display', 'lat': 40.87684369079793...",376,10463,US,Bronx,NY,United States,"[5500 Broadway (230th Street), Bronx, NY 10463...","[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],NaN,NaN
2,e-0-4b79cc46f964a520c5122fe3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4b79cc46f964a520c5122fe3,Tibbett Diner,3033 Tibbett Ave,btwn 230th & 231st,40.880404,-73.908937,"[{'label': 'display', 'lat': 40.8804044222466,...",452,10463,US,Bronx,NY,United States,"[3033 Tibbett Ave (btwn 230th & 231st), Bronx,...","[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],NaN,NaN
3,e-0-55f81cd2498ee903149fcc64-3,0,"[{'summary': 'This spot is popular', 'type': '...",55f81cd2498ee903149fcc64,Starbucks,171 W 230th St,Kimberly Pl,40.877531,-73.905582,"[{'label': 'display', 'lat': 40.87753134921497...",441,10463,US,Bronx,NY,United States,"[171 W 230th St (Kimberly Pl), Bronx, NY 10463...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],NaN,NaN
4,e-0-4b5357adf964a520319827e3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4b5357adf964a520319827e3,Dunkin',5501 Broadway,W 230th St,40.877136,-73.906666,"[{'label': 'display', 'lat': 40.87713584201589...",342,10463,US,Bronx,NY,United States,"[5501 Broadway (W 230th St), Bronx, NY 10463, ...","[{'id': '4bf58dd8d48988d148941735', 'name': 'D...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],NaN,NaN
5,e-0-4b88e053f964a5208a1132e3-5,0,"[{'summary': 'This spot is popular', 'type': '...",4b88e053f964a5208a1132e3,Rite Aid,5237 Broadway,228th Street,40.875467,-73.908906,"[{'label': 'display', 'lat': 40.87546657443470...",190,10463,US,Bronx,NY,United States,"[5237 Broadway (228th Street), Bronx, NY 10463...","[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],NaN,NaN
6,e-0-4a725fa1f964a520f6da1fe3-6,0,"[{'summary': 'This spot is popular', 'type': '...",4a725fa1f964a520f6da1fe3,TCR The Club of Riverdale,2600 Netherland Ave,NaN,40.878628,-73.914568,"[{'label': 'display', 'lat': 40.8786283, 'lng'...",402,10463,US,Bronx,NY,United States,"[2600 Netherland Ave, Bronx, NY 10463, United ...","[{'id': '4e39a891bd410d7aed40cbc2', 'name': 'T...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],40358759,NaN
7,e-0-4b9c9c6af964a520b27236e3-7,0,"[{'summary': 'This spot is popular', 'type': '...",4b9c9c6af964a520b27236e3,Land & Sea Restaurant,5535 Broadway,231st St,40.877885,-73.905873,"[{'label': 'display', 'lat': 40.87788463309788...",429,10463,US,Bronx,NY,United States,"[5535 Broadway (231st St), Bronx, NY 10463, Un...","[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,0,[],NaN,NaN
8,e-0-57655be738faa6

In [24]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Arturo's,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",40.874412,-73.910271
1,Bikram Yoga,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",40.876844,-73.906204
2,Tibbett Diner,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",40.880404,-73.908937
3,Starbucks,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",40.877531,-73.905582
4,Dunkin',"[{'id': '4bf58dd8d48988d148941735', 'name': 'D...",40.877136,-73.906666
5,Rite Aid,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",40.875467,-73.908906
6,TCR The Club of Riverdale,"[{'id': '4e39a891bd410d7aed40cbc2', 'name': 'T...",40.878628,-73.914568
7,Land & Sea Restaurant,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",40.877885,-73.905873
8,Starbucks,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",40.873755,-73.908613
9,Astral Fitness & Wellness Center,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",40.876705,-73.906372


In [25]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

In [26]:
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin',Donut Shop,40.877136,-73.906666


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

27 venues were returned by Foursquare.


In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            Limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
# type your answer here

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [30]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,59,59,59,59,59,59
Carnegie Hill,83,83,83,83,83,83
Central Harlem,45,45,45,45,45,45
Chelsea,98,98,98,98,98,98
Chinatown,100,100,100,100,100,100
Civic Center,86,86,86,86,86,86
Clinton,100,100,100,100,100,100
East Harlem,44,44,44,44,44,44
East Village,100,100,100,100,100,100


In [31]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 325 uniques categories.


In [32]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soup Place,South American Restaurant,Souther

In [33]:
manhattan_onehot.shape

(2997, 326)

In [34]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soup Place,South American Restaurant,Souther

In [35]:
manhattan_grouped.shape

(40, 326)

In [36]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.10
1          Hotel  0.07
2  Memorial Site  0.05
3  Boat or Ferry  0.05
4            Gym  0.05


----Carnegie Hill----
                  venue  freq
0           Coffee Shop  0.08
1           Yoga Studio  0.04
2  Gym / Fitness Center  0.04
3             Bookstore  0.04
4           Pizza Place  0.04


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1  American Restaurant  0.04
2                  Bar  0.04
3   Seafood Restaurant  0.04
4       Cosmetics Shop  0.04


----Chelsea----
                venue  freq
0         Art Gallery  0.16
1         Coffee Shop  0.06
2  Italian Restaurant  0.03
3      Ice Cream Shop  0.03
4            Boutique  0.02


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.08
1               Bakery  0.04
2         Cocktail Bar  0.04
3   Salon / Barbershop  0.03
4  American Restaurant  0.03


----Civic Center----
               venue  freq
0  

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Boat or Ferry,Gym,Memorial Site,Coffee Shop,Plaza,Playground,Wine Shop,Shopping Mall
1,Carnegie Hill,Coffee Shop,Yoga Studio,Café,Japanese Restaurant,Gym,Grocery Store,Bookstore,Gym / Fitness Center,Pizza Place,Wine Shop
2,Central Harlem,African Restaurant,Chinese Restaurant,American Restaurant,Bar,Cosmetics Shop,French Restaurant,Seafood Restaurant,Caribbean Restaurant,Market,Bookstore
3,Chelsea,Art Gallery,Coffee Shop,Ice Cream Shop,Italian Restaurant,Café,Boutique,Park,Cycle Studio,Theater,Market
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Optical Shop,Salon / Barbershop,Coffee Shop,Spa,American Restaurant,Dim Sum Restaurant,Dumpling Restaurant


In [39]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:10]

array([2, 1, 0, 1, 0, 2, 2, 4, 1, 2], dtype=int32)

In [40]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()# check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Sandwich Place,Gym,American Restaurant,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bakery,Cocktail Bar,Optical Shop,Salon / Barbershop,Coffee Shop,Spa,American Restaurant,Dim Sum Restaurant,Dumpling Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Bakery,Chinese Restaurant,Mobile Phone Shop,Grocery Store,Pizza Place,Donut Shop,Mexican Restaurant,Bank,Tapas Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,4,Mexican Restaurant,Café,Pizza Place,Lounge,Restaurant,American Restaurant,Wine Bar,Spanish Restaurant,Deli / Bodega,Bakery
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Sushi Restaurant,Cocktail Bar,Bakery,Sandwich Place,Yoga Studio


In [42]:
manhattan_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Sandwich Place,Gym,American Restaurant,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bakery,Cocktail Bar,Optical Shop,Salon / Barbershop,Coffee Shop,Spa,American Restaurant,Dim Sum Restaurant,Dumpling Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Bakery,Chinese Restaurant,Mobile Phone Shop,Grocery Store,Pizza Place,Donut Shop,Mexican Restaurant,Bank,Tapas Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,4,Mexican Restaurant,Café,Pizza Place,Lounge,Restaurant,American Restaurant,Wine Bar,Spanish Restaurant,Deli / Bodega,Bakery
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Sushi Restaurant,Cocktail Bar,Bakery,Sandwich Place,Yoga Studio
5,Manhattan,Manhattanville,40.816934,-73.957385,4,Coffee Shop,Seafood Restaurant,Mexican Restaurant,Chinese Restaurant,Italian Restaurant,Park,Supermarket,Bike Trail,Lounge,Falafel Restaurant
6,Manhattan,Central Harlem,40.815976,-73.943211,0,African Restaurant,Chinese Restaurant,American Restaurant,Bar,Cosmetics Shop,French Restaurant,Seafood Restaurant,Caribbean Restaurant,Market,Bookstore
7,Manhattan,East Harlem,40.792249,-73.944182,4,Mexican Restaurant,Deli / Bodega,Bakery,Thai Restaurant,Latin American Restaurant,Chinese Restaurant,Gas Station,Sandwich Place,Grocery Store,Street Art
8,Manhattan,Upper East Side,40.775639,-73.960508,1,Italian Restaurant,Gym / Fitness Center,Bakery,Juice Bar,Yoga Studio,Wine Shop,Hotel,Exhibit,French Restaurant,Sushi Restaurant
9,Manhattan,Yorkville,40.775930,-73.947118,1,Coffee Shop,Italian Restaurant,Gym,Sushi Restaurant,Deli / Bodega,Bar,Wine Shop,Mexican Restaurant,Diner,Japanese Restaurant


In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [44]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Optical Shop,Salon / Barbershop,Coffee Shop,Spa,American Restaurant,Dim Sum Restaurant,Dumpling Restaurant
6,Central Harlem,African Restaurant,Chinese Restaurant,American Restaurant,Bar,Cosmetics Shop,French Restaurant,Seafood Restaurant,Caribbean Restaurant,Market,Bookstore
20,Lower East Side,Chinese Restaurant,Art Gallery,Café,Cocktail Bar,Theater,Yoga Studio,Bakery,Juice Bar,Tennis Court,Filipino Restaurant
22,Little Italy,Chinese Restaurant,Italian Restaurant,Bakery,Mediterranean Restaurant,Spa,Hotel,Thai Restaurant,Bubble Tea Shop,Café,Japanese Restaurant


In [45]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Italian Restaurant,Gym / Fitness Center,Bakery,Juice Bar,Yoga Studio,Wine Shop,Hotel,Exhibit,French Restaurant,Sushi Restaurant
9,Yorkville,Coffee Shop,Italian Restaurant,Gym,Sushi Restaurant,Deli / Bodega,Bar,Wine Shop,Mexican Restaurant,Diner,Japanese Restaurant
10,Lenox Hill,Italian Restaurant,Coffee Shop,Pizza Place,Café,Cocktail Bar,Sushi Restaurant,Gym,Gym / Fitness Center,Burger Joint,Salad Place
12,Upper West Side,Italian Restaurant,Bakery,Wine Bar,Coffee Shop,Seafood Restaurant,Bookstore,Dessert Shop,Middle Eastern Restaurant,American Restaurant,Pizza Place
17,Chelsea,Art Gallery,Coffee Shop,Ice Cream Shop,Italian Restaurant,Café,Boutique,Park,Cycle Studio,Theater,Market
18,Greenwich Village,Italian Restaurant,Coffee Shop,Gym,Pizza Place,Ice Cream Shop,Bakery,Wine Bar,Gourmet Shop,Caribbean Restaurant,Park
19,East Village,Pizza Place,Coffee Shop,Cocktail Bar,Mexican Restaurant,Japanese Restaurant,Wine Bar,Juice Bar,Ice Cream Shop,Vietnamese Restaurant,Ramen Restaurant
21,Tribeca,Park,Italian Restaurant,Wine Bar,Spa,Café,Poke Place,Playground,Bakery,Coffee Shop,Men's Store
23,Soho,Italian Restaurant,Mediterranean Restaurant,Gym,Coffee Shop,Bakery,Clothing Store,Spa,Dessert Shop,Café,Art Gallery
24,West Village,Coffee Shop,Wine Bar,Italian Restaurant,Park,Jazz Club,American Restaurant,Seafood Restaurant,Bakery,New American Restaurant,Pizza Place


In [46]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Lincoln Square,Café,Italian Restaurant,Gym / Fitness Center,Plaza,Performing Arts Venue,Concert Hall,Theater,Wine Shop,American Restaurant,Indie Movie Theater
14,Clinton,Theater,Gym / Fitness Center,Coffee Shop,Wine Shop,Gym,Hotel,Italian Restaurant,Sandwich Place,Pizza Place,Spa
15,Midtown,Coffee Shop,Clothing Store,Hotel,Theater,Pizza Place,Cuban Restaurant,Spa,Salon / Barbershop,Sporting Goods Shop,Men's Store
16,Murray Hill,Sandwich Place,Hotel,Coffee Shop,Pizza Place,Steakhouse,Gym / Fitness Center,Japanese Restaurant,Chinese Restaurant,Jewish Restaurant,Bar
28,Battery Park City,Park,Hotel,Boat or Ferry,Gym,Memorial Site,Coffee Shop,Plaza,Playground,Wine Shop,Shopping Mall
29,Financial District,Coffee Shop,Hotel,Café,American Restaurant,Pizza Place,Park,Gym,Sandwich Place,Salad Place,Cocktail Bar
32,Civic Center,Spa,Park,Coffee Shop,French Restaurant,Cocktail Bar,Yoga Studio,Sushi Restaurant,Gym / Fitness Center,Hotel,American Restaurant
33,Midtown South,Korean Restaurant,Hotel,Japanese Restaurant,Burger Joint,Hotel Bar,Café,Dessert Shop,Lounge,Spa,New American Restaurant
38,Flatiron,Gym / Fitness Center,Italian Restaurant,Park,Cosmetics Shop,American Restaurant,Salon / Barbershop,Wine Shop,Outdoor Sculpture,Cycle Studio,New American Restaurant
39,Hudson Yards,Hotel,Café,Gym / Fitness Center,Italian Restaurant,American Restaurant,Gym,Coffee Shop,Restaurant,Dog Run,Park


In [47]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Roosevelt Island,Park,Bubble Tea Shop,Dry Cleaner,Coffee Shop,Greek Restaurant,Outdoors & Recreation,Liquor Store,Sandwich Place,Scenic Lookout,Gym / Fitness Center
37,Stuyvesant Town,Park,Boat or Ferry,Baseball Field,Gym / Fitness Center,German Restaurant,Harbor / Marina,Cocktail Bar,Coffee Shop,Bistro,Farmers Market


In [48]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Gym,American Restaurant,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant
2,Washington Heights,Café,Bakery,Chinese Restaurant,Mobile Phone Shop,Grocery Store,Pizza Place,Donut Shop,Mexican Restaurant,Bank,Tapas Restaurant
3,Inwood,Mexican Restaurant,Café,Pizza Place,Lounge,Restaurant,American Restaurant,Wine Bar,Spanish Restaurant,Deli / Bodega,Bakery
4,Hamilton Heights,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Sushi Restaurant,Cocktail Bar,Bakery,Sandwich Place,Yoga Studio
5,Manhattanville,Coffee Shop,Seafood Restaurant,Mexican Restaurant,Chinese Restaurant,Italian Restaurant,Park,Supermarket,Bike Trail,Lounge,Falafel Restaurant
7,East Harlem,Mexican Restaurant,Deli / Bodega,Bakery,Thai Restaurant,Latin American Restaurant,Chinese Restaurant,Gas Station,Sandwich Place,Grocery Store,Street Art
26,Morningside Heights,Park,Bookstore,American Restaurant,Coffee Shop,Paper / Office Supplies Store,Burger Joint,Tennis Court,Pizza Place,Deli / Bodega,Outdoor Sculpture
36,Tudor City,Café,Park,Mexican Restaurant,Pizza Place,Deli / Bodega,Dog Run,Asian Restaurant,Garden,Diner,Thai Restaurant
